# Abundance of taxa on nose and severity

In [219]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import biom
from biom import load_table
import numpy as np
from scipy.stats import spearmanr
from scipy.stats import mannwhitneyu
from statsmodels.stats.multitest import multipletests
import itertools
import matplotlib.patches
from scipy.stats import pearsonr



In [220]:
# Load the metadata
metadata_path = '../Data/Metadata/updated_clean_ant_skin_metadata_microbiome_type.tab'
metadata = pd.read_csv(metadata_path, sep='\t')

metadata['#sample-id'] = metadata['#sample-id'].str.replace('_', '')
# Set Sample-ID as the index for the metadata dataframe 
metadata = metadata.set_index('#sample-id')


metadata

,PlateNumber,PlateLocation,i5,i5Sequence,i7,i7Sequence,identifier,Sequence,Plate ID,Well location,...,enrolment_date,enrolment_season,hiv_exposure,hiv_status,household_size,o_scorad,FWD_filepath,REV_filepath,group,microbiome_type
#sample-id,,,,,,,,,,,,,,,,,,,,,
Ca009STL,1,A1,SA501,ATCGTACG,SA701,CGAGAGTT,SA701SA501,CGAGAGTT-ATCGTACG,1.010000e+21,A1,...,4/16/2015,Autumn,Unexposed,negative,4.0,40,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,skin-ADL,skin
900221,1,B1,SA502,ACTATCTG,SA701,CGAGAGTT,SA701SA502,CGAGAGTT-ACTATCTG,1.010000e+21,B1,...,8/11/2015,Winter,Unexposed,negative,7.0,34,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,skin-ADL,skin
Ca010EBL,1,C1,SA503,TAGCGAGT,SA701,CGAGAGTT,SA701SA503,CGAGAGTT-TAGCGAGT,1.010000e+21,C1,...,11/20/2014,Spring,Unexposed,negative,7.0,21,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,skin-ADL,skin
900460,1,D1,SA504,CTGCGTGT,SA701,CGAGAGTT,SA701SA504,CGAGAGTT-CTGCGTGT,1.010000e+21,D1,...,9/23/2015,Spring,Unexposed,NaN,4.0,40,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,skin-ADL,skin
900051,1,E1,SA505,TCATCGAG,SA701,CGAGAGTT,SA701SA505,CGAGAGTT-TCATCGAG,1.010000e+21,E1,...,4/21/2015,Autumn,Unexposed,negative,7.0,41,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,skin-ADL,skin
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Ca006ONL2,6,H1,SA508,GACACCGT,SB701,CTCGACTT,SB701SA508,CTCGACTT-GACACCGT,1.010000e+21,H1,...,3/25/2015,Autumn,Unexposed,negative,3.0,34,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,skin-ADL,skin
Ca006ONNL,6,F2,SA506,CGTGAGTG,SB702,CGAAGTAT,SB702SA506,CGAAGTAT-CGTGAGTG,1.010000e+21,F2,...,3/25/2015,Autumn,Unexposed,negative,3.0,34,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,skin-ADNL,skin
Ca006ONNL2,6,H2,SA508,GACACCGT,SB702,CGAAGTAT,SB702SA508,CGAAGTAT-GACACCGT,1.010000e+21,H2,...,3/25/2015,Autumn,Unexposed,negative,3.0,34,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,/Users/yac027/Gallo_lab/16S_AD_Dube_Dupont/ato...,skin-ADNL,skin


In [221]:
# Read in table at ASV level
# biom_path = '../Data/Tables/Absolute_Abundance_Tables/feature_table_with_tax_labels_Genus.biom'
biom_path = '../Data/Tables/Absolute_Abundance_Tables/209766_filtered_by_prevalence_1pct_rare_Genus-ASV-non-collapse.biom'
# biom_path = '../Data/Tables/Absolute_Abundance_Tables/209766_filtered_by_prevalence_1pct_rare_Genus.biom'

biom_tbl = load_table(biom_path)
df = pd.DataFrame(biom_tbl.to_dataframe().T)

# delete the prefix from the index
df.index = df.index.str.replace('15564.', '')

# Get samples that start with 'nares' in metadata
nares_samples = metadata[metadata['group'].str.startswith('nares')].index

# Subset to only certain area samples
cape_town_samples = metadata[metadata['area'].str.startswith('Umtata')].index
umtata_town_samples = metadata[metadata['area'].str.startswith('Umtata')].index

# Filter df to keep only nares samples
df = df.loc[nares_samples]

df

,g__Streptococcus_ASV-1,g__Streptococcus_ASV-2,g__Corynebacterium_ASV-1,g__Corynebacterium_ASV-4,g___ASV-5,g__Cutibacterium_ASV-1,g___ASV-33,g___ASV-30,g___ASV-22,g__Cutibacterium_ASV-2,...,g__Copromonas_ASV-3,g__Microvirga_ASV-1,g__Leptotrichia_A_993758_ASV-17,g___ASV-103,g__Leptotrichia_A_993758_ASV-13,g__Capnocytophaga_820688_ASV-8,g__UBA6175_ASV-2,g__Comamonas_F_589250_ASV-3,g__Bosea_ASV-2,g___ASV-145
#sample-id,,,,,,,,,,,,,,,,,,,,,
Ca009STPN,51.0,0,240.0,0,0,0,2.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
900223,190.0,0,899.0,0,0,0,2.0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Ca010EBPN,208.0,0,154.0,0,177.0,38.0,0,0,0,19.0,...,0,0,0,0,0,0,0,0,0,0
900462,532.0,484.0,254.0,200.0,104.0,11.0,0,0,0,8.0,...,0,0,0,0,0,0,0,0,0,0
900053,2478.0,0,0,0,3.0,7.0,0,0,0,1.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9003992,328.0,0,1463.0,892.0,15.0,72.0,0,0,0,59.0,...,0,0,0,0,0,0,0,0,0,0
900402,156.0,0,0,0,0,32.0,0,0,0,19.0,...,0,0,0,0,0,0,0,0,0,0
9004022,126.0,0,241.0,0,235.0,14.0,0,0,4.0,7.0,...,0,0,0,0,0,0,0,0,0,0


In [222]:
# # Extract base taxa names before _ASV
# base_taxa = [col.split('_ASV')[0] if '_ASV' in col else col for col in df.columns]

# # Create dictionary mapping base taxa to list of original column names
# taxa_groups = {}
# for base, col in zip(base_taxa, df.columns):
#     if base not in taxa_groups:
#         taxa_groups[base] = []
#     taxa_groups[base].append(col)

# # Sum columns within each taxa group
# combined_df = pd.DataFrame()
# for base_taxon, columns in taxa_groups.items():
#     if len(columns) > 1:  # Only combine if multiple ASVs exist
#         combined_df[base_taxon] = df[columns].sum(axis=1)
#     else:
#         combined_df[base_taxon] = df[columns[0]]

# # Replace original df with combined version
# df = combined_df
# df

In [223]:
def rclr_transform(df, pseudocount=1e-6):
    """
    Applies Robust Centered Log-Ratio (RCLR) transformation to a DataFrame.
    Zeros are ignored in the geometric mean calculation per sample.
    """
    # Convert sparse to dense if needed
    df = df.sparse.to_dense()

    # Replace 0 with np.nan to ignore in log and mean
    df_masked = df.replace(0, np.nan)

    # Apply log (with pseudocount only where needed)
    log_df = np.log(df_masked + pseudocount)

    # Subtract mean of each row (feature)
    rclr_df = log_df.sub(log_df.mean(axis=1, skipna=True), axis=0)

    return rclr_df



rclr_df = rclr_transform(df)

In [224]:
# Map the 'group' column from metadata to df based on matching index
rclr_df['group'] = metadata.loc[rclr_df.index, 'group']
rclr_df

,g__Streptococcus_ASV-1,g__Streptococcus_ASV-2,g__Corynebacterium_ASV-1,g__Corynebacterium_ASV-4,g___ASV-5,g__Cutibacterium_ASV-1,g___ASV-33,g___ASV-30,g___ASV-22,g__Cutibacterium_ASV-2,...,g__Microvirga_ASV-1,g__Leptotrichia_A_993758_ASV-17,g___ASV-103,g__Leptotrichia_A_993758_ASV-13,g__Capnocytophaga_820688_ASV-8,g__UBA6175_ASV-2,g__Comamonas_F_589250_ASV-3,g__Bosea_ASV-2,g___ASV-145,group
#sample-id,,,,,,,,,,,,,,,,,,,,,
Ca009STPN,2.024339,NaN,3.573153,NaN,NaN,NaN,-1.214339,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD
900223,0.489140,NaN,2.043399,NaN,NaN,NaN,-4.064737,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD
Ca010EBPN,2.783217,NaN,2.482631,NaN,2.621828,1.083265,NaN,NaN,NaN,0.390118,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD
900462,4.109607,4.015048,3.370298,3.131281,2.477354,0.230859,NaN,NaN,NaN,-0.087595,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD
900053,5.869346,NaN,NaN,NaN,-0.847248,0.000050,NaN,NaN,NaN,-1.945860,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9003992,3.255158,NaN,4.750389,4.255610,0.170195,1.738810,NaN,NaN,NaN,1.539682,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD
900402,2.275817,NaN,NaN,NaN,NaN,0.691697,NaN,NaN,NaN,0.170400,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD
9004022,2.428674,NaN,3.077189,NaN,3.051978,0.231450,NaN,NaN,-1.021313,-0.461697,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD


In [225]:
rclr_df['group'].value_counts()

# rclr_df = rclr_df.rename(columns={
#     col: col.replace("g__Haemophilus_D_734546", "g__Haemophilus")
#     for col in df.columns
#     if col.startswith("g__Haemophilus_D_734546")
# })


# rclr_df = rclr_df.rename(columns={
#     col: col.replace("g__Veillonella_A", "g__Veillonella")
#     for col in df.columns
#     if col.startswith("g__Veillonella_A")
# })



group
nares-AD    108
nares-H      89
Name: count, dtype: int64

In [226]:
def pairwise_mannwhitney_bh(df, feature, group_col='group'):
    groups = df[group_col].unique()
    comparisons = list(itertools.combinations(groups, 2))

    raw_pvals = []
    labels = []
    pairs = []

    for g1, g2 in comparisons:
        x = df[df[group_col] == g1][feature].dropna()
        y = df[df[group_col] == g2][feature].dropna()
        _, p = mannwhitneyu(x, y, alternative='two-sided')
        raw_pvals.append(p)
        labels.append(f"{g1} vs {g2}")
        pairs.append((g1, g2))

    _, pvals_corrected, _, _ = multipletests(raw_pvals, method='fdr_bh')

    results_df = pd.DataFrame({
        'Comparison': labels,
        'Pair': pairs,
        'Raw p-value': raw_pvals,
        'BH-corrected p-value': pvals_corrected
    })

    return results_df

In [227]:
# Filter for nares samples
nares_samples = rclr_df[rclr_df['group'].isin(['nares-AD', 'nares-H'])]
nares_samples

,g__Streptococcus_ASV-1,g__Streptococcus_ASV-2,g__Corynebacterium_ASV-1,g__Corynebacterium_ASV-4,g___ASV-5,g__Cutibacterium_ASV-1,g___ASV-33,g___ASV-30,g___ASV-22,g__Cutibacterium_ASV-2,...,g__Microvirga_ASV-1,g__Leptotrichia_A_993758_ASV-17,g___ASV-103,g__Leptotrichia_A_993758_ASV-13,g__Capnocytophaga_820688_ASV-8,g__UBA6175_ASV-2,g__Comamonas_F_589250_ASV-3,g__Bosea_ASV-2,g___ASV-145,group
#sample-id,,,,,,,,,,,,,,,,,,,,,
Ca009STPN,2.024339,NaN,3.573153,NaN,NaN,NaN,-1.214339,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD
900223,0.489140,NaN,2.043399,NaN,NaN,NaN,-4.064737,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD
Ca010EBPN,2.783217,NaN,2.482631,NaN,2.621828,1.083265,NaN,NaN,NaN,0.390118,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD
900462,4.109607,4.015048,3.370298,3.131281,2.477354,0.230859,NaN,NaN,NaN,-0.087595,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD
900053,5.869346,NaN,NaN,NaN,-0.847248,0.000050,NaN,NaN,NaN,-1.945860,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9003992,3.255158,NaN,4.750389,4.255610,0.170195,1.738810,NaN,NaN,NaN,1.539682,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD
900402,2.275817,NaN,NaN,NaN,NaN,0.691697,NaN,NaN,NaN,0.170400,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD
9004022,2.428674,NaN,3.077189,NaN,3.051978,0.231450,NaN,NaN,-1.021313,-0.461697,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD


In [228]:
def pairwise_mannwhitney_bh(df, feature, group_col='group'):
    groups = df[group_col].unique()
    comparisons = list(itertools.combinations(groups, 2))

    raw_pvals = []
    labels = []
    pairs = []

    for g1, g2 in comparisons:
        x = df[df[group_col] == g1][feature].dropna()
        y = df[df[group_col] == g2][feature].dropna()

        # Skip comparison if either group has no data
        if len(x) == 0 or len(y) == 0:
            continue

        _, p = mannwhitneyu(x, y, alternative='two-sided')
        raw_pvals.append(p)
        labels.append(f"{g1} vs {g2}")
        pairs.append((g1, g2))

    if len(raw_pvals) == 0:
        return pd.DataFrame(columns=['Comparison', 'Pair', 'Raw p-value', 'BH-corrected p-value'])

    _, pvals_corrected, _, _ = multipletests(raw_pvals, method='fdr_bh')

    results_df = pd.DataFrame({
        'Comparison': labels,
        'Pair': pairs,
        'Raw p-value': raw_pvals,
        'BH-corrected p-value': pvals_corrected
    })

    return results_df


In [229]:
# Define custom color palette
group_palette = {
    'nares-H': '#ADD8E6',     # baby blue
    'nares-AD': '#E31A1C'    # red
}

strip_palette = {
    'nares-H': '#6CA6CD',     # darker baby blue
    'nares-AD': '#A50000'    # darker red
}


In [230]:
def plot_multi_taxa_boxplots(data, taxa_list, group_palette, strip_palette, title_name, order=['nares-H', 'nares-AD']):
    n_taxa = len(taxa_list)
    group_count = len(order)

    # Prepare long format dataframe
    plot_data = []
    for i, taxon in enumerate(taxa_list):
        for j, group in enumerate(order):
            xpos = i * group_count + j
            values = data.loc[data['group'] == group, taxon]
            for v in values:
                plot_data.append({
                    'x': xpos,
                    'taxon': taxon.replace(' g__', '').split('_')[0],
                    'value': v,
                    'group': group
                })

    plot_df = pd.DataFrame(plot_data)

    # Plot
    fig, ax = plt.subplots(figsize=(n_taxa * 2.5, 5))

    sns.boxplot(
        data=plot_df,
        x='x',
        y='value',
        hue='group',
        palette=group_palette,
        width=0.5,
        fliersize=0,
        dodge=False,
        ax=ax
    )

    sns.stripplot(
        data=plot_df,
        x='x',
        y='value',
        hue='group',
        palette=strip_palette,
        dodge=False,
        jitter=True,
        size=4,
        alpha=0.75,
        ax=ax
    )

    # Remove duplicated legends
    handles, labels = ax.get_legend_handles_labels()

    # Count samples per group
    group_counts = {g: data[data['group'] == g].shape[0] for g in order}

    # Create updated legend labels
    legend_labels = [f"{g.replace('skin-', '')} (n={group_counts[g]})" for g in order]

    # Add updated legend — horizontally at bottom-left
    ax.legend(
        handles[:3],
        legend_labels,
        # title='Sample',
        loc='lower left',
        bbox_to_anchor=(0.35, -0.35),  # adjust y for spacing below plot
        ncol=len(order),            # show horizontally
        fontsize=12,
        title_fontsize=10,
        frameon=True,
        borderaxespad=0
    )


    # X-tick labels per taxon group
    xtick_positions = [i * group_count + 1 for i in range(n_taxa)]
    # xtick_labels = [taxon.replace('g__', '').split('_')[0] for taxon in taxa_list]
    xtick_labels = [taxon.replace('g__', '').replace('_ASV-', '\nASV-') for taxon in taxa_list]
    ax.set_xticks(xtick_positions)
    ax.set_xticklabels(xtick_labels, fontsize=14)

    ax.set_xlim(-0.5, n_taxa * group_count - 0.5)
    ax.set_ylabel("RCLR-transformed abundance", fontsize=14)
    ax.tick_params(axis='y', labelsize=14)
    ax.set_xlabel(" ")

    ax.set_title(title_name, fontsize=18, pad=14)

    # ------------------------
    # Annotate p-values
    # ------------------------
    for i, taxon in enumerate(taxa_list):
        stats = pairwise_mannwhitney_bh(data, taxon)
        y_max = data[taxon].max()
        offset = y_max * 0.15

        for k, row in stats.iterrows():
            g1, g2 = row['Pair']
            pval = row['BH-corrected p-value']
            xpos1 = i * group_count + order.index(g1)
            xpos2 = i * group_count + order.index(g2)
            x = (xpos1 + xpos2) / 2
            y = y_max + offset * k

            stars = '***' if pval < 0.001 else '**' if pval < 0.01 else '*' if pval < 0.05 else ''
            label = f"{pval:.1e} {stars}"

            ax.plot([xpos1, xpos1, xpos2, xpos2], [y - 0.01, y, y, y - 0.01], lw=1, color='black')
            ax.text(x, y + 0.01, label, ha='center', va='bottom', fontsize=12)

    sns.despine()
    fig.tight_layout()
    return fig

In [231]:
taxa_list = ['g__Streptococcus_ASV-1', 'g__Streptococcus_ASV-2', 'g__Staphylococcus_ASV-1',
            'g__Micrococcus_ASV-1', 'g__Veillonella_A_ASV-1', 'g__Haemophilus_D_734546_ASV-1']

# taxa_list = [' g__Staphylococcus', ' g__Streptococcus',
#             ' g__Micrococcus', ' g__Veillonella_A', ' g__Haemophilus_D_734546']

# taxa_list = ['g__Streptococcus_ASV-1']

fig = plot_multi_taxa_boxplots(nares_samples, taxa_list, group_palette, strip_palette, 'Differential Nares Taxa of Umtata Samples')
fig.savefig('../Plots/Analysis_figures/RCLR_Abundance/rclr_multi_taxa_boxplot_all_taxa_ASV_nares_Umtata.png', dpi=600)


/Users/yangchen/miniforge3/envs/qiime2-metagenome-2024.10/lib/python3.10/site-packages/seaborn/_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):
/Users/yangchen/miniforge3/envs/qiime2-metagenome-2024.10/lib/python3.10/site-packages/seaborn/_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):
/Users/yangchen/miniforge3/envs/qiime2-metagenome-2024.10/lib/python3.10/site-packages/seaborn/_oldcore.py:1075: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  data_subset = grouped_data.get_group(pd_key)


## Show correlation with lesion severity

In [232]:
# Map o_scorad from metadata to nares_samples based on matching indexes
nares_samples = nares_samples.merge(metadata[['o_scorad']], left_index=True, right_index=True)
# Map area from metadata to nares_samples based on matching indexes
nares_samples = nares_samples.merge(metadata[['area']], left_index=True, right_index=True)

# Filter to only include samples from Umtata
nares_samples = nares_samples[nares_samples['area'] == 'Umtata']

nares_samples

,g__Streptococcus_ASV-1,g__Streptococcus_ASV-2,g__Corynebacterium_ASV-1,g__Corynebacterium_ASV-4,g___ASV-5,g__Cutibacterium_ASV-1,g___ASV-33,g___ASV-30,g___ASV-22,g__Cutibacterium_ASV-2,...,g___ASV-103,g__Leptotrichia_A_993758_ASV-13,g__Capnocytophaga_820688_ASV-8,g__UBA6175_ASV-2,g__Comamonas_F_589250_ASV-3,g__Bosea_ASV-2,g___ASV-145,group,o_scorad,area
#sample-id,,,,,,,,,,,,,,,,,,,,,
900223,0.489140,NaN,2.043399,NaN,NaN,NaN,-4.064737,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD,34,Umtata
900462,4.109607,4.015048,3.370298,3.131281,2.477354,0.230859,NaN,NaN,NaN,-0.087595,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD,40,Umtata
900228,5.928436,4.161521,4.350885,NaN,NaN,-1.483926,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD,34,Umtata
900231,2.717672,2.194084,1.846542,1.309399,1.453980,-1.398651,NaN,NaN,NaN,-1.804116,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD,23,Umtata
900242,1.392800,NaN,2.082003,NaN,NaN,NaN,-2.352774,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-H,NaN,Umtata
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9003962,1.564364,NaN,5.630157,NaN,-2.242297,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD,28,Umtata
900399,0.250958,0.056802,NaN,NaN,-0.502814,3.381324,NaN,NaN,NaN,2.993694,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD,54,Umtata
9003992,3.255158,NaN,4.750389,4.255610,0.170195,1.738810,NaN,NaN,NaN,1.539682,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD,54,Umtata


In [233]:
# Convert 'o_scorad' to numeric (coerce errors to NaN)
rclr_df['o_scorad'] = pd.to_numeric(metadata['o_scorad'], errors='coerce')

# Drop rows with missing SCORAD values
rclr_df = rclr_df.dropna(subset=['o_scorad'])
rclr_df


,g__Streptococcus_ASV-1,g__Streptococcus_ASV-2,g__Corynebacterium_ASV-1,g__Corynebacterium_ASV-4,g___ASV-5,g__Cutibacterium_ASV-1,g___ASV-33,g___ASV-30,g___ASV-22,g__Cutibacterium_ASV-2,...,g__Leptotrichia_A_993758_ASV-17,g___ASV-103,g__Leptotrichia_A_993758_ASV-13,g__Capnocytophaga_820688_ASV-8,g__UBA6175_ASV-2,g__Comamonas_F_589250_ASV-3,g__Bosea_ASV-2,g___ASV-145,group,o_scorad
#sample-id,,,,,,,,,,,,,,,,,,,,,
Ca009STPN,2.024339,NaN,3.573153,NaN,NaN,NaN,-1.214339,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD,40.0
900223,0.489140,NaN,2.043399,NaN,NaN,NaN,-4.064737,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD,34.0
Ca010EBPN,2.783217,NaN,2.482631,NaN,2.621828,1.083265,NaN,NaN,NaN,0.390118,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD,21.0
900462,4.109607,4.015048,3.370298,3.131281,2.477354,0.230859,NaN,NaN,NaN,-0.087595,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD,40.0
900053,5.869346,NaN,NaN,NaN,-0.847248,0.000050,NaN,NaN,NaN,-1.945860,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD,41.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
900399,0.250958,0.056802,NaN,NaN,-0.502814,3.381324,NaN,NaN,NaN,2.993694,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD,54.0
9003992,3.255158,NaN,4.750389,4.255610,0.170195,1.738810,NaN,NaN,NaN,1.539682,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD,54.0
9004022,2.428674,NaN,3.077189,NaN,3.051978,0.231450,NaN,NaN,-1.021313,-0.461697,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD,38.0


In [234]:
# Filter to only include skin-ADL samples for severity correlation analysis
rclr_df = rclr_df[rclr_df['group'] == 'nares-AD']
rclr_df

,g__Streptococcus_ASV-1,g__Streptococcus_ASV-2,g__Corynebacterium_ASV-1,g__Corynebacterium_ASV-4,g___ASV-5,g__Cutibacterium_ASV-1,g___ASV-33,g___ASV-30,g___ASV-22,g__Cutibacterium_ASV-2,...,g__Leptotrichia_A_993758_ASV-17,g___ASV-103,g__Leptotrichia_A_993758_ASV-13,g__Capnocytophaga_820688_ASV-8,g__UBA6175_ASV-2,g__Comamonas_F_589250_ASV-3,g__Bosea_ASV-2,g___ASV-145,group,o_scorad
#sample-id,,,,,,,,,,,,,,,,,,,,,
Ca009STPN,2.024339,NaN,3.573153,NaN,NaN,NaN,-1.214339,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD,40.0
900223,0.489140,NaN,2.043399,NaN,NaN,NaN,-4.064737,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD,34.0
Ca010EBPN,2.783217,NaN,2.482631,NaN,2.621828,1.083265,NaN,NaN,NaN,0.390118,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD,21.0
900462,4.109607,4.015048,3.370298,3.131281,2.477354,0.230859,NaN,NaN,NaN,-0.087595,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD,40.0
900053,5.869346,NaN,NaN,NaN,-0.847248,0.000050,NaN,NaN,NaN,-1.945860,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD,41.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
900399,0.250958,0.056802,NaN,NaN,-0.502814,3.381324,NaN,NaN,NaN,2.993694,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD,54.0
9003992,3.255158,NaN,4.750389,4.255610,0.170195,1.738810,NaN,NaN,NaN,1.539682,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD,54.0
9004022,2.428674,NaN,3.077189,NaN,3.051978,0.231450,NaN,NaN,-1.021313,-0.461697,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nares-AD,38.0


In [235]:
def plot_combined_box_and_severity(data, taxa_list, group_palette, strip_palette, title_name, order=['nares-H', 'nares-AD']):
    n_taxa = len(taxa_list)
    fig, axes = plt.subplots(
        nrows=n_taxa,
        ncols=2,
        figsize=(6, 3 * n_taxa),
        sharey='row',
        gridspec_kw={'width_ratios': [2, 1.5]}
    )

    if n_taxa == 1:
        axes = np.array([axes])

    title_y_positions = [0.91, 0.73, 0.55, 0.36, 0.175]

    for i, taxon in enumerate(taxa_list):
        ax_box = axes[i, 0] if n_taxa > 1 else axes[0]
        ax_corr = axes[i, 1] if n_taxa > 1 else axes[1]

        # --- Boxplot data ---
        df_box = data[['group', taxon]].copy()
        df_box = df_box[df_box['group'].isin(order)]
        df_box['group'] = pd.Categorical(df_box['group'], categories=order, ordered=True)
        df_box = df_box.dropna(subset=[taxon])

        if df_box.empty:
            print(f"⚠️ Skipping {taxon} — no non-missing data.")
            ax_box.set_visible(False)
            ax_corr.set_visible(False)
            continue

        df_box['taxon'] = taxon.replace('g__', '').replace('_ASV-', ' ASV-')

        # --- Correlation data ---
        df_corr = data[data['group'].isin(['nares-H', 'nares-AD'])][['o_scorad', taxon]].copy()
        df_corr[taxon] = pd.to_numeric(df_corr[taxon], errors='coerce')
        df_corr['o_scorad'] = pd.to_numeric(df_corr['o_scorad'], errors='coerce')
        df_corr = df_corr.dropna()

        # --- Left: Boxplot ---
        sns.boxplot(
            data=df_box,
            x='group',
            y=taxon,
            order=order,
            palette=group_palette,
            ax=ax_box,
            width=0.5,
            fliersize=0
        )
        sns.stripplot(
            data=df_box,
            x='group',
            y=taxon,
            order=order,
            palette=strip_palette,
            ax=ax_box,
            jitter=True,
            size=4,
            alpha=0.7
        )
        ax_box.set_ylabel("RCLR Abundance", fontsize=14)
        ax_box.set_xlabel("")
        ax_box.set_xticklabels(['H', 'AD'], fontsize=14)
        ax_box.tick_params(axis='y', labelsize=10)

        # --- Add Mann-Whitney U p-value ---
        if len(order) == 2:
            group1_vals = df_box[df_box['group'] == order[0]][taxon]
            group2_vals = df_box[df_box['group'] == order[1]][taxon]
            if not group1_vals.empty and not group2_vals.empty:
                stat, pval = mannwhitneyu(group1_vals, group2_vals, alternative='two-sided')
                
                # Use minimum value instead of max
                y_line = 6.2  # 👈 your manual y-position for the bracket line
                y_text = 7  # 👈 your manual y-position for the p-value text

                # Horizontal line (bracket)
                ax_box.plot([0, 0, 1, 1],
                            [y_line, y_line - 0.03, y_line - 0.03, y_line],
                            color='black')

                # P-value text
                stars = '***' if pval < 0.001 else '**' if pval < 0.01 else '*' if pval < 0.05 else ''
                ax_box.text(0.5, y_text, f"{pval:.1e} {stars}", ha='center', va='top', fontsize=12)


        # --- Right: Correlation with severity ---
        ax_corr.set_visible(True)
        if not df_corr.empty:
            # Calculate correlation
            r, pval = pearsonr(df_corr['o_scorad'], df_corr[taxon])

            # Choose dot color
            if pval > 0.05:
                dot_color = 'grey'
            elif taxon == 'g__Staphylococcus_ASV-1':
                dot_color = 'salmon'
            else:
                dot_color = None

            # Plot
            sns.regplot(
                data=df_corr,
                x='o_scorad',
                y=taxon,
                scatter_kws={'alpha': 0.5, 's': 20, 'color': dot_color} if dot_color else {'alpha': 0.5, 's': 20},
                line_kws={'color': 'black'},
                ax=ax_corr
            )

            # Add correlation label
            r_label = f"Pearson r = {r:.2f}\np = {pval:.1e}"
            ax_corr.text(0.05, 0.90, r_label, transform=ax_corr.transAxes,
                        fontsize=12, va='top', ha='left',
                        bbox=dict(facecolor='white', alpha=0.7, edgecolor='none'))

            ax_corr.set_xlabel("SCORAD", fontsize=12)

        # --- Title per taxon ---
        if i < len(title_y_positions):
            fig.text(0.5, title_y_positions[i], df_box['taxon'].iloc[0], ha='center', fontsize=16)

    fig.suptitle(title_name, fontsize=18, y=0.97)
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.subplots_adjust(hspace=0.45)
    sns.despine()

    return fig

In [236]:
# taxa_list = ['g__Staphylococcus_ASV-1', 'g__Streptococcus_ASV-1', 'g__Streptococcus_ASV-2', 'g__Micrococcus_ASV-1', 'g__Veillonella_A_ASV-1']
taxa_list = ['g__Staphylococcus_ASV-1', 'g__Streptococcus_ASV-1', 'g__Streptococcus_ASV-2', 'g__Micrococcus_ASV-1', 'g__Veillonella_A_ASV-1']

# taxa_list = [' g__Staphylococcus', ' g__Streptococcus', ' g__Micrococcus', ' g__Veillonella_A', ' g__Haemophilus_D_734546']

fig = plot_combined_box_and_severity(
    data=nares_samples,
    taxa_list=taxa_list,
    group_palette=group_palette,
    strip_palette=strip_palette,
    title_name='\nNares of Umtata Children'
)
fig.savefig('../Plots/Analysis_figures/Severity_Correlations/combined_box_scatter_Umtata_nares.png', dpi=600)


/Users/yangchen/miniforge3/envs/qiime2-metagenome-2024.10/lib/python3.10/site-packages/seaborn/categorical.py:641: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_vals = vals.groupby(grouper)
/var/folders/22/yck9vwx53w1c38tvj_c0_tz00000gn/T/ipykernel_7373/1981148617.py:51: FutureWarning: Passing `palette` without assigning `hue` is deprecated.
  sns.stripplot(
/Users/yangchen/miniforge3/envs/qiime2-metagenome-2024.10/lib/python3.10/site-packages/seaborn/_oldcore.py:1119: FutureWarning: use_inf_as_na option is deprecated and will be removed in a future version. Convert inf values to NaN before operating instead.
  with pd.option_context('mode.use_inf_as_na', True):
/Users/yangchen/miniforge3/envs/qiime2-metagenome-2024.10/lib/python3.10/site-packages/seaborn/_oldcore.py:1119: FutureWarning: u